In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

## ETL

En este caso trabajas en una empresa de venta al por menor de productos italianos y debes realizar la limpieza, transformación e integración de datos de ventas, productos y clientes para su análisis.

1. Lectura de la Información:

    - Leer los archivos CSV (ventas.csv, productos.csv, clientes.csv).

    - Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

2. Transformación de Datos:

    - Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.

    - Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

    - Aplicar transformaciones relevantes según sea necesario: por ejemplo, convertir tipos de datos, renombrar columnas, crear nuevas características derivadas, etc.

In [2]:
with open("files/productos.csv", 'r') as file:

    lines = file.readlines()

In [13]:
print(lines[8])

H8,Pesto Genovese,Salsas,5.99,Italia,Una salsa italiana de albahaca, piñones, queso parmesano y aceite de oliva, perfecta para pastas.



In [38]:
## revisar:
def unificar(lines):
    for line in lines:
        split = line.split(",")
        join = ",".join(split[5:])
        all = split[0:5]
        all.append(join)
        print(all)
        #df_all = pd.DataFrame(all)
        #return df_all.to_csv("files/productos1.csv")

In [39]:
unificar(lines)

['ID', 'Nombre_Producto', 'Categoría', 'Precio', 'Origen', 'Descripción\n']
['A1', 'Pizza Margherita', 'Platos Preparados', '8.99', 'Italia', 'Clásica pizza italiana con tomate, mozzarella fresca y albahaca.\n']
['B2', 'Risotto de Champiñones', 'Platos Preparados', '6.75', 'Italia', 'Risotto cremoso con champiñones frescos, una delicia italiana para disfrutar en casa.\n']
['C3', 'Tiramisú', 'Postres', '5.49', 'Italia', 'Postre clásico italiano con capas de bizcocho de café, mascarpone y cacao en polvo.\n']
['D4', 'Panettone', 'Repostería', '10.99', 'Italia', 'Pan dulce navideño italiano con frutas confitadas y pasas, un regalo tradicional en Italia.\n']
['E5', 'Orecchiette', 'Productos Secos', '4.29', 'Italia', 'Pequeñas pastas con forma de orecchiette ideales para salsas de tomate o pesto.\n']
['F6', 'Polenta Tradicional', 'Harinas', '3.75', 'Italia', 'Harina de maíz italiana para preparar la tradicional polenta, perfecta como acompañamiento.\n']
['G7', 'Mozzarella di Bufala', 'Produc

In [28]:
split = lines[8].split(",")
join = ",".join(split[5:])
all = split[0:5]
all.append(join)
print(all)


['H8', 'Pesto Genovese', 'Salsas', '5.99', 'Italia', 'Una salsa italiana de albahaca, piñones, queso parmesano y aceite de oliva, perfecta para pastas.\n']


In [104]:
customer = pd.read_csv("files/clientes.csv").reset_index(inplace = True)
sales = customer = pd.read_csv("files/ventas.csv", index_col = 0)
items = pd.read_csv("files/productos.csv", index_col = 0, error_bad_lines=False, warn_bad_lines=False)



In [109]:
def apertura_fichero(ruta):
    csv = pd.read_csv(ruta)
    return csv.reset_index(inplace = True)


In [110]:
apertura_fichero("files/clientes.csv")

In [107]:
# se puede revisar y mejorar
def exploracion_datos(csv):
    forma = csv.shape
    print(f"La forma es {forma}")
    print("_______________")
    nulos = csv.isna().sum()
    print(f"Los nulos son: {nulos}")
    print("_______________")
    duplicados = csv.duplicated().sum()
    print(f"Hay {duplicados} duplicados")
    print("_______________")
    tipo_dato = csv.dtypes
    print(f"Los datos son de tipo {tipo_dato}")
    print("_______________")
    columnas = csv.columns
    print(f"Las columnas son {columnas}")
    print("_______________")
    print(csv.head())
    

    


In [108]:
exploracion_datos(customer)

La forma es (100, 6)
_______________
Los nulos son: index          0
ID_Cliente     0
ID_Producto    0
Fecha_Venta    0
Cantidad       0
Total          0
dtype: int64
_______________
Hay 0 duplicados
_______________
Los datos son de tipo index            int64
ID_Cliente       int64
ID_Producto     object
Fecha_Venta     object
Cantidad         int64
Total          float64
dtype: object
_______________
Las columnas son Index(['index', 'ID_Cliente', 'ID_Producto', 'Fecha_Venta', 'Cantidad',
       'Total'],
      dtype='object')
_______________
   index  ID_Cliente ID_Producto Fecha_Venta  Cantidad  Total
0      0         723          A1  2023-11-22         2  17.98
1      1         498          C3  2023-11-21         1   5.49
2      2         121          D4  2023-11-20         3  32.97
3      3         885         L12  2023-11-19         1   6.49
4      4         347         Q17  2023-11-18         2   7.98


In [106]:
customer.reset_index(inplace = True)

In [105]:
customer.head()

,ID_Producto,Fecha_Venta,Cantidad,Total
ID_Cliente,,,,
723,A1,2023-11-22,2,17.98
498,C3,2023-11-21,1,5.49
121,D4,2023-11-20,3,32.97
885,L12,2023-11-19,1,6.49
347,Q17,2023-11-18,2,7.98


In [81]:
exploracion_datos(items)

La forma es (21, 6)
_______________
Los nulos son: ID                 0
Nombre_Producto    0
Categoría          0
Precio             0
Origen             0
Descripción        1
dtype: int64
_______________
Hay 0 duplicados
_______________
Los datos son de tipo ID                  object
Nombre_Producto     object
Categoría          float64
Precio              object
Origen              object
Descripción         object
dtype: object
_______________
Las columnas son Index(['ID', 'Nombre_Producto', 'Categoría', 'Precio', 'Origen',
       'Descripción'],
      dtype='object')
_______________
                        ID    Nombre_Producto  Categoría  Precio  \
A1        Pizza Margherita  Platos Preparados       8.99  Italia   
B2  Risotto de Champiñones  Platos Preparados       6.75  Italia   
C3                Tiramisú            Postres       5.49  Italia   
D4               Panettone         Repostería      10.99  Italia   
E5             Orecchiette    Productos Secos       4.29  Italia

In [70]:
exploracion_datos(sales)

La forma es (100, 4)
_______________
Los nulos son: ID_Producto    0
Fecha_Venta    0
Cantidad       0
Total          0
dtype: int64
_______________
Hay 0 duplicados
_______________
Los datos son de tipo ID_Producto     object
Fecha_Venta     object
Cantidad         int64
Total          float64
dtype: object
_______________
Las columnas son Index(['ID_Producto', 'Fecha_Venta', 'Cantidad', 'Total'], dtype='object')
